<a href="https://colab.research.google.com/github/kridtapon/MultiFrame-Momentum/blob/main/MultiFrame_Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.1 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_window, adjust=False).mean()
    return macd_line, signal_line

# Define the stock symbol and time period
symbol = 'META'  # Example stock symbol
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
weekly_df = yf.download(symbol, start=start_date, end=end_date, interval='1wk')
weekly_df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
daily_df = yf.download(symbol, start=start_date, end=end_date, interval='1d')
daily_df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate MACD for weekly (HTF) and daily (LTF)
weekly_df['MACD'], weekly_df['Signal'] = calculate_macd(weekly_df)
daily_df['MACD'], daily_df['Signal'] = calculate_macd(daily_df)

# Forward-fill weekly MACD to daily
weekly_df = weekly_df.reindex(daily_df.index, method='ffill')

daily_df['HTF_Trend'] = weekly_df['MACD'] > weekly_df['Signal']  # Weekly trend confirmation

# Entry Condition: Weekly MACD bullish & Daily MACD crossover bullish
daily_df['Entry'] = (daily_df['HTF_Trend']) & (daily_df['MACD'] > daily_df['Signal'])

# Exit Condition: Weekly MACD bearish & Daily MACD crossover bearish
daily_df['Exit'] = (~daily_df['HTF_Trend']) & (daily_df['MACD'] < daily_df['Signal'])

# Filter data for backtesting (2020-2025)
daily_df = daily_df[(daily_df.index.year >= 2020) & (daily_df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=daily_df['Close'],
    entries=daily_df['Entry'],
    exits=daily_df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           359831.973084
Total Return [%]                       259.831973
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                        3673.14519
Max Drawdown [%]                        53.917953
Max Drawdown Duration                       411.0
Total Trades                                    9
Total Closed Trades                             9
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            55.555556
Best Trade [%]                         175.478353
Worst Trade [%]                        -41.926466
Avg Winning Trade [%]                   59.352028
Avg Losing Trade [%]                   -15.597765


In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_window, adjust=False).mean()
    return macd_line, signal_line

# Define the stock symbol and time period
symbol = 'META'  # Example stock symbol
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the daily data
daily_df = yf.download(symbol, start=start_date, end=end_date)
daily_df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Resample the daily data to weekly data (OHLCV: Open, High, Low, Close, Volume)
weekly_df = daily_df.resample('W').agg({
    'Open': 'first',  # First value in the week for Open
    'High': 'max',    # Highest value in the week for High
    'Low': 'min',     # Lowest value in the week for Low
    'Close': 'last',  # Last value in the week for Close
    'Volume': 'sum'   # Total volume for the week
})

# Calculate MACD for weekly (HTF) and daily (LTF)
weekly_df['MACD'], weekly_df['Signal'] = calculate_macd(weekly_df)
daily_df['MACD'], daily_df['Signal'] = calculate_macd(daily_df)

# Forward-fill weekly MACD to daily
weekly_df = weekly_df.reindex(daily_df.index, method='ffill')

daily_df['HTF_Trend'] = weekly_df['MACD'] > weekly_df['Signal']  # Weekly trend confirmation

# Entry Condition: Weekly MACD bullish & Daily MACD crossover bullish
daily_df['Entry'] = (daily_df['HTF_Trend']) & (daily_df['MACD'] > daily_df['Signal'])

# Exit Condition: Weekly MACD bearish & Daily MACD crossover bearish
daily_df['Exit'] = (~daily_df['HTF_Trend']) & (daily_df['MACD'] < daily_df['Signal'])

# Filter data for backtesting (2020-2025)
daily_df = daily_df[(daily_df.index.year >= 2020) & (daily_df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=daily_df['Close'],
    entries=daily_df['Entry'],
    exits=daily_df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           177154.171798
Total Return [%]                        77.154172
Benchmark Return [%]                   180.172876
Max Gross Exposure [%]                      100.0
Total Fees Paid                       2588.440417
Max Drawdown [%]                        58.836884
Max Drawdown Duration                       445.0
Total Trades                                   10
Total Closed Trades                            10
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 40.0
Best Trade [%]                         136.451597
Worst Trade [%]                        -40.418369
Avg Winning Trade [%]                   50.154906
Avg Losing Trade [%]                   -12.163194
